In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Set plot style
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams['font.size'] = 12
%config InlineBackend.figure_format = 'retina'

# Load cleaned data
Data is preprocessed in the [data preprocessing](./create_dataset.ipynb) notebook. This includes concatenating the data, removing outliers, missing values, and irrelevant columns and generating relevant features based on given data.

In [ ]:
data_per_day = pd.read_pickle('../data/processed/data_one_day_clean.pickle')

In [ ]:
data_per_day.info()

In [ ]:
# convert date columns from object to datetime
date_cols = ['Zeitstempel', 'Sicherheitsbestand wird erreicht am', 'Meldebestand wird erreicht am']
for col in date_cols:
    data_per_day[col] = pd.to_datetime(data_per_day[col])

In [ ]:
data_per_day.head()

# Build target variable "Verbrauch" per day
Difference of the "Füllstand" the current day to the next day. This is the oil consumption per day. Implementation also see [data preprocessing](./create_dataset.ipynb), calculate "Füllstand" difference from day before to current day, then shift by one day to the past, since this difference is the consumption of the day before.

In [ ]:
#!IMPORTANT! Shift the consumption by one day to the future, since on day x the "Füllstand" of day x+1 is not known, necessary for calculating the consumption of day x
for id in data_per_day["Tank-ID"].unique():
    temp_df = pd.DataFrame(data_per_day[data_per_day["Tank-ID"] == id])
    temp_df["Verbrauch"] = temp_df["Füllstand"].diff()

# Concat matching historical weather data
We implemented a wrapper to gain historical and forecast weather data per day using an open source API, see in [weather data](../src/api/weather.py) notebook. This then can be easily concatenated as an external feature to the data.

In [ ]:
from src.api import WeatherAPI

weather_api = WeatherAPI()

merged_dfs = []
for id in data_per_day["Tank-ID"].unique():
    temp_df = data_per_day[data_per_day["Tank-ID"] == id]

    # get attributes
    latitude = temp_df["Längengrad"].iloc[0]
    longitude = temp_df["Breitengrad"].iloc[0]
    if temp_df["Zeitstempel"].dtypes == 'object':
        temp_df["Zeitstempel"] = pd.to_datetime(temp_df["Zeitstempel"])
    start_date = temp_df["Zeitstempel"].min().strftime("%Y-%m-%d")
    end_date = temp_df["Zeitstempel"].max().strftime("%Y-%m-%d")
    print("Start date:", start_date, "End date:", end_date, "Day difference:", (temp_df["Zeitstempel"].max() - temp_df["Zeitstempel"].min()).days)

    # get matching weather data
    weather_data = weather_api.get_data(latitude, longitude, start_date, end_date)

    # remove timezone information
    weather_data['date'] = weather_data['date'].dt.tz_localize(None)
    weather_data = weather_data.rename(columns={'date': 'Zeitstempel'})

    # join
    print("Data shape before merge:", temp_df.shape)
    temp_df = temp_df.merge(weather_data, on=['Zeitstempel'], how='left')
    print("Data shape after merge:", temp_df.shape)
    display(temp_df.head(5))

    # append
    merged_dfs.append(temp_df)

In [ ]:
merged_data = pd.concat(merged_dfs)

In [ ]:
print("Data shape before merging weatehr data:", data_per_day.shape)
print("Data shape after merging weather data:", merged_data.shape)

In [ ]:
# set index to "Zeitstempel"
df = merged_data.set_index('Zeitstempel')

# Investigate data

In [ ]:
# plot "Füllstand" time series based on tank ID
fig, ax = plt.subplots(figsize=(12, 6))
for tank_id in df["Tank-ID"].unique():
    df[df["Tank-ID"] == tank_id]["Füllstand"].plot(ax=ax, label=f'Tank {tank_id}')
plt.xlabel('Date')
plt.ylabel('Füllstand')
plt.title('"Füllstand" over time')
plt.legend()
plt.show()

Since the Tank ID 5 has a different pattern than the other tanks, and the data is not complete, we will drop this tank.

In [ ]:
# drop irrelevant columns
cols_to_drop = ["Sicherheitsbestand wird erreicht am", "Meldebestand wird erreicht am"] # "Füllstand"
df = df.drop(cols_to_drop, axis=1)
# drop ID 5
df = df[df['Tank-ID'] != 5]

In [ ]:
# plot target "Verbrauch" time series based on tank ID
fig, axes = plt.subplots(2, 1, figsize=(12, 8))
for tank_id in df["Tank-ID"].unique():
    df[df["Tank-ID"] == tank_id]["Verbrauch"].plot(ax=axes[0], label=f'Tank {tank_id}')
    df[df["Tank-ID"] == tank_id]["Verbrauch smoothed"].plot(ax=axes[1], label=f'Tank {tank_id}')

axes[0].set_title('Consumption over time')
axes[1].set_title('Consumption over time (smoothed)')

for ax in axes:
    ax.set_xlabel('Date')
    ax.set_ylabel('Verbrauch')
    ax.legend()

plt.tight_layout()
plt.show()

Since outliers above 0 make no sense and also in our implemented smoothing functionality it is detected as outliers we will set a maximum value of 0 for the consumption. Logically also one can not consume oil and the tank gets filled up by consuming it. This is a clear indicator for a wrong measurement. In the plot above we can see that the consumption is mostly below 0, where a value of -10 indicates a consumption of 10 liters, the negative sign is due to the resulting decline in the tank by consumption. Thus the 0 value indicates a full no consumption and thus no decline in the tank. For better understanding we will further take the absolute value of the consumption. after correcting the outliers due to sensor errors.
--> 1. Set consumption to 0 if above 0
--> 2. Take absolute values

In [ ]:
# correct outliers
df.loc[df['Verbrauch'] > 0, 'Verbrauch'] = 0.0
# take absolute values
df['Verbrauch'] = df['Verbrauch'].abs()

In [ ]:
# check if outliers are corrected
fig, ax = plt.subplots(figsize=(12, 4))
for tank_id in df["Tank-ID"].unique():
    df[df["Tank-ID"] == tank_id]["Verbrauch"].plot(ax=ax, label=f'Tank {tank_id}')
plt.xlabel('Date')
plt.ylabel('Verbrauch')
plt.title('Consumption over time')
plt.legend()
plt.show()

# Prepare Train and Test Data
For forcasting following steps are necessary:
1. Split data into train and test data
2. Normalize data
3. (Create sequences of data)
5. Split data into X and y
8. Save data

In [ ]:
# split data into train and test data, based on tank ID
train_data = df[df['Tank-ID'] != 2]
test_data = df[df['Tank-ID'] == 2]

# get x and y values
X_train = train_data.drop('Verbrauch', axis=1).values
y_train = train_data['Verbrauch'].values
X_test = test_data.drop('Verbrauch', axis=1).values
y_test = test_data['Verbrauch'].values

In [ ]:
# save data
np.save('../data/processed/X_train.npy', X_train)
np.save('../data/processed/y_train.npy', y_train)
np.save('../data/processed/X_test.npy', X_test)
np.save('../data/processed/y_test.npy', y_test)

In [ ]:
# normalize data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Feature Selection
* via correlation
* via feature importance


## Option 1: ARIMA model


Reasoning:
* As seen in the plot of the target variable "Verbrauch" over time above, we can see that the oil consumption has a clear trend and seasonality:
* Peaks in winter (high consumption for heating), dips in summer (lower demand), and rises again in autumn. The pattern is cyclical, reflecting higher oil use in colder months and lower in warmer months.

This suggests that the time series is not stationary and will require differencing to make it stationary, at least a difference order of 1.

--> This is a good indicator for using an ARIMA model.

In [ ]:
# Plotting autocorrelation and partial autocorrelation for an extensive set of time series lags, hue as the id
# Function to calculate autocorrelation
def autocorr_plot(df, lags=30):
    plt.figure(figsize=(10, 6))

    # Loop over each unique ID and plot autocorrelation for each
    for id_value in df['id'].unique():
        subset = df[df['id'] == id_value]
        plot_acf(subset['value'], lags=lags, alpha=0.05, title=f'Autocorrelation for ID {id_value}', ax=plt.gca())

    plt.legend(df['id'].unique(), title='ID')
    plt.show()

# Call the function with your dataframe
autocorr_plot(df, lags=30)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt

# fit model
model = ARIMA(y_train, order=(5,1,0))

model_fit = model.fit()

# make prediction
yhat = model_fit.forecast(steps=len(y_test))

# evaluate forecast (Best: RMSE = 0, Worst: RMSE = 1)
rmse = sqrt(mean_squared_error(y_test, yhat))
print('Test RMSE: %.3f' % rmse)

## Option 2: Prophet model


## Option 3: Simple ML model
